# SK-7-MultiModal : Images, Audio et Vision

**Navigation** : [<< 06-ProcessFramework](06-SemanticKernel-ProcessFramework.ipynb) | [Index](README.md) | [08-MCP >>](08-SemanticKernel-MCP.ipynb)

---

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Generer des **images** avec DALL-E 3
2. Analyser des **images** avec GPT-4 Vision
3. Transcrire de l'**audio** avec Whisper
4. Generer de l'**audio** avec Text-to-Speech
5. Combiner les modalites dans un **pipeline**

### Prerequis

- Python 3.10+
- Notebooks 01-06 completes
- Cle API OpenAI configuree (`.env`)
- Acces aux modeles DALL-E et Whisper (OpenAI payant)

### Duree estimee : 45 minutes

---

## Sommaire

| Section | Contenu | Concepts cles |
|---------|---------|---------------|
| 1 | Introduction | Capacites multi-modales |
| 2 | Text-to-Image | DALL-E 3 |
| 3 | Image-to-Text | GPT-4 Vision |
| 4 | Speech-to-Text | Whisper |
| 5 | Text-to-Speech | OpenAI TTS |
| 6 | Pipeline combine | Audio -> Texte -> Image |
| 7 | Conclusion | Resume, exercices |

> **Multi-Modal AI** : Les modeles modernes peuvent traiter et generer du contenu dans plusieurs modalites (texte, image, audio, video). SK fournit des connecteurs uniformes pour ces capacites.

In [ ]:
# Installation
%pip install semantic-kernel python-dotenv openai Pillow --quiet

import os
from dotenv import load_dotenv
from semantic_kernel import Kernel

load_dotenv()

# Verification de la cle API
api_key = os.getenv("OPENAI_API_KEY")
print(f"API Key configuree: {'Oui' if api_key else 'Non'}")

## 1. Introduction aux capacites Multi-Modales

SK supporte plusieurs modalites via des services specialises :

| Modalite | Direction | Service SK | Modele OpenAI |
|----------|-----------|------------|---------------|
| **Texte -> Image** | Generation | `OpenAITextToImage` | DALL-E 3 |
| **Image -> Texte** | Analyse | `OpenAIChatCompletion` | GPT-4 Vision |
| **Audio -> Texte** | Transcription | `OpenAIAudioToText` | Whisper |
| **Texte -> Audio** | Synthese | `OpenAITextToAudio` | TTS-1, TTS-1-HD |

### Architecture multi-modale

```
┌─────────────────────────────────────────────────────┐
│                    KERNEL                           │
│  ┌─────────────┐  ┌─────────────┐  ┌────────────┐  │
│  │    Chat     │  │   Image     │  │   Audio    │  │
│  │  Completion │  │ Generation  │  │ Services   │  │
│  │   (GPT-4V)  │  │  (DALL-E)   │  │ (Whisper)  │  │
│  └─────────────┘  └─────────────┘  └────────────┘  │
│         ↑               ↑               ↑          │
│         └───────────────┼───────────────┘          │
│                         │                          │
│              Unified Service Interface             │
└─────────────────────────────────────────────────────┘
```

## 2. Text-to-Image avec DALL-E 3

Generation d'images a partir de descriptions textuelles.

In [ ]:
from semantic_kernel.connectors.ai.open_ai import OpenAITextToImage
from IPython.display import Image, display
import urllib.request

# Configuration du service DALL-E
kernel = Kernel()

dalle_service = OpenAITextToImage(
    service_id="dalle",
    ai_model_id="dall-e-3"  # ou "dall-e-2" pour moins cher
)
kernel.add_service(dalle_service)

print("Service DALL-E configure")
print("\nModeles disponibles:")
print("| Modele    | Resolution | Prix/image |")
print("|-----------|------------|------------|")
print("| dall-e-3  | 1024x1024  | $0.04      |")
print("| dall-e-3  | 1792x1024  | $0.08      |")
print("| dall-e-2  | 1024x1024  | $0.02      |")

In [ ]:
# Generation d'une image
prompt = "A futuristic city with flying cars and neon lights, cyberpunk style, highly detailed"

print(f"Prompt: {prompt}")
print("Generation en cours...")

try:
    # Generer l'image
    image_result = await dalle_service.generate_image(
        description=prompt,
        width=1024,
        height=1024
    )
    
    print(f"\nImage generee avec succes!")
    print(f"URL: {image_result[:80]}...")
    
    # Afficher l'image
    display(Image(url=image_result))
    
except Exception as e:
    print(f"Erreur: {e}")
    print("Assurez-vous d'avoir acces a l'API DALL-E.")

### Interpretation : DALL-E 3

DALL-E 3 offre des ameliorations significatives :

| Caracteristique | DALL-E 2 | DALL-E 3 |
|-----------------|----------|----------|
| **Qualite** | Bonne | Excellente |
| **Texte dans images** | Difficile | Supporte |
| **Suivi du prompt** | Moyen | Tres precis |
| **Styles** | Limites | Variés |
| **Prix** | $0.02 | $0.04-0.08 |

**Bonnes pratiques pour les prompts** :
- Etre specifique sur le style ("cyberpunk", "watercolor", "photorealistic")
- Inclure des details de composition
- Specifier l'eclairage et l'ambiance

## 3. Image-to-Text avec GPT-4 Vision

Analyse et description d'images.

In [ ]:
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import ChatHistory, ImageContent, TextContent

# Configuration du service GPT-4 Vision
vision_service = OpenAIChatCompletion(
    service_id="vision",
    ai_model_id="gpt-4o"  # ou "gpt-4o-mini" pour moins cher
)
kernel.add_service(vision_service)

print("Service GPT-4 Vision configure")
print("\nModeles avec vision:")
print("| Modele       | Vision | Prix (input/1M) |")
print("|--------------|--------|-----------------|")
print("| gpt-4o       | Oui    | $2.50           |")
print("| gpt-4o-mini  | Oui    | $0.15           |")
print("| gpt-4-turbo  | Oui    | $10.00          |")

In [ ]:
# Analyse d'une image
# Utilisons une image publique pour l'exemple
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/1200px-Cat03.jpg"

print(f"Image a analyser: {image_url[:50]}...")
display(Image(url=image_url, width=300))

# Creer l'historique avec image
history = ChatHistory()
history.add_user_message([
    TextContent(text="Decris cette image en detail. Que vois-tu?"),
    ImageContent(uri=image_url)
])

try:
    response = await vision_service.get_chat_message_contents(
        chat_history=history
    )
    
    print("\nDescription de l'image:")
    print("-" * 40)
    print(str(response[0]))
    
except Exception as e:
    print(f"Erreur: {e}")

### Interpretation : GPT-4 Vision

Les modeles avec vision peuvent :

| Capacite | Description | Exemple |
|----------|-------------|--------|
| **Description** | Decrire le contenu | "Un chat roux assis..." |
| **OCR** | Lire du texte | Extraire du texte d'une photo |
| **Analyse** | Comprendre le contexte | "Cette image montre une scene de bureau" |
| **Comparaison** | Comparer des images | Differences entre 2 versions |
| **Code** | Lire du code/diagrammes | Comprendre un schema UML |

**Limitations** :
- Pas de generation d'images (utiliser DALL-E)
- Cout par token d'image (~85 tokens pour 512x512)

## 4. Speech-to-Text avec Whisper

Transcription audio en texte.

In [ ]:
from semantic_kernel.connectors.ai.open_ai import OpenAIAudioToText

# Configuration du service Whisper
whisper_service = OpenAIAudioToText(
    service_id="whisper",
    ai_model_id="whisper-1"
)
kernel.add_service(whisper_service)

print("Service Whisper configure")
print("\nCaracteristiques Whisper:")
print("| Caracteristique | Valeur           |")
print("|-----------------|------------------|")
print("| Langues         | 99+              |")
print("| Formats audio   | mp3, wav, m4a... |")
print("| Taille max      | 25 MB            |")
print("| Prix            | $0.006/minute    |")

In [ ]:
# Exemple de transcription (necessite un fichier audio)
# Creeons un fichier audio de test avec TTS d'abord

from openai import AsyncOpenAI
import tempfile
import os

async def create_test_audio():
    """Cree un fichier audio de test avec TTS."""
    client = AsyncOpenAI()
    
    text = "Bonjour, ceci est un test de transcription audio avec Whisper."
    
    response = await client.audio.speech.create(
        model="tts-1",
        voice="alloy",
        input=text
    )
    
    # Sauvegarder temporairement
    temp_path = os.path.join(tempfile.gettempdir(), "test_audio.mp3")
    with open(temp_path, "wb") as f:
        f.write(response.content)
    
    return temp_path, text

try:
    audio_path, original_text = await create_test_audio()
    print(f"Audio cree: {audio_path}")
    print(f"Texte original: {original_text}")
    
    # Transcription
    with open(audio_path, "rb") as audio_file:
        transcription = await whisper_service.get_text_content(
            audio_content=audio_file.read(),
            settings=None
        )
    
    print(f"\nTranscription: {transcription}")
    
except Exception as e:
    print(f"Erreur: {e}")
    print("\nExemple conceptuel (fichier audio requis):")
    print("""```python
with open('audio.mp3', 'rb') as f:
    text = await whisper_service.get_text_content(audio_content=f.read())
print(text)
```""")

## 5. Text-to-Speech

Generation audio a partir de texte.

In [ ]:
from semantic_kernel.connectors.ai.open_ai import OpenAITextToAudio
from IPython.display import Audio

# Configuration du service TTS
tts_service = OpenAITextToAudio(
    service_id="tts",
    ai_model_id="tts-1"  # ou "tts-1-hd" pour haute qualite
)
kernel.add_service(tts_service)

print("Service TTS configure")
print("\nVoix disponibles:")
print("| Voix   | Description        |")
print("|--------|-------------------|")
print("| alloy  | Neutre, versatile |")
print("| echo   | Grave, masculin   |")
print("| fable  | Narratif, chaleureux |")
print("| onyx   | Profond, autoritaire |")
print("| nova   | Feminin, dynamique |")
print("| shimmer| Doux, expressif   |")

In [ ]:
# Generation audio
text_to_speak = "Semantic Kernel est un SDK puissant pour creer des applications d'intelligence artificielle."

try:
    audio_content = await tts_service.get_audio_content(
        text=text_to_speak,
        settings=None
    )
    
    # Sauvegarder et jouer
    output_path = os.path.join(tempfile.gettempdir(), "output_speech.mp3")
    with open(output_path, "wb") as f:
        f.write(audio_content.data)
    
    print(f"Audio genere: {output_path}")
    print(f"Taille: {len(audio_content.data)} bytes")
    
    # Afficher le lecteur audio
    display(Audio(output_path))
    
except Exception as e:
    print(f"Erreur: {e}")
    print("\nExemple conceptuel:")
    print("""```python
audio = await tts_service.get_audio_content(text="Bonjour!")
with open('speech.mp3', 'wb') as f:
    f.write(audio.data)
```""")

## 6. Pipeline Multi-Modal

Combinons les modalites dans un flux complet.

In [ ]:
async def multimodal_pipeline(audio_description: str):
    """
    Pipeline multi-modal:
    1. Texte -> Description enrichie (LLM)
    2. Description -> Image (DALL-E)
    3. Image -> Analyse (Vision)
    4. Analyse -> Audio (TTS)
    """
    
    print("=" * 60)
    print("PIPELINE MULTI-MODAL")
    print("=" * 60)
    
    # Etape 1: Enrichir la description
    print("\n[1/4] Enrichissement de la description...")
    chat_service = kernel.get_service(service_id="vision")
    history = ChatHistory()
    history.add_user_message(
        f"Transforme cette description en prompt detaille pour DALL-E: '{audio_description}'"
    )
    response = await chat_service.get_chat_message_contents(chat_history=history)
    enriched_prompt = str(response[0])
    print(f"Prompt enrichi: {enriched_prompt[:100]}...")
    
    # Etape 2: Generer l'image
    print("\n[2/4] Generation de l'image...")
    dalle = kernel.get_service(service_id="dalle")
    image_url = await dalle.generate_image(
        description=enriched_prompt,
        width=1024,
        height=1024
    )
    print(f"Image generee: {image_url[:50]}...")
    display(Image(url=image_url, width=400))
    
    # Etape 3: Analyser l'image
    print("\n[3/4] Analyse de l'image...")
    history = ChatHistory()
    history.add_user_message([
        TextContent(text="Decris cette image de facon poetique en 2 phrases."),
        ImageContent(uri=image_url)
    ])
    response = await chat_service.get_chat_message_contents(chat_history=history)
    analysis = str(response[0])
    print(f"Analyse: {analysis}")
    
    # Etape 4: Convertir en audio
    print("\n[4/4] Generation audio...")
    tts = kernel.get_service(service_id="tts")
    audio = await tts.get_audio_content(text=analysis)
    
    output_path = os.path.join(tempfile.gettempdir(), "pipeline_output.mp3")
    with open(output_path, "wb") as f:
        f.write(audio.data)
    
    print("\n" + "=" * 60)
    print("PIPELINE TERMINE")
    print("=" * 60)
    
    display(Audio(output_path))
    
    return {
        "original": audio_description,
        "enriched_prompt": enriched_prompt,
        "image_url": image_url,
        "analysis": analysis,
        "audio_path": output_path
    }

# Test du pipeline
try:
    result = await multimodal_pipeline("Un paysage de montagne au coucher du soleil")
except Exception as e:
    print(f"Pipeline interrompu: {e}")
    print("\nCe pipeline necessite l'acces a DALL-E, GPT-4V, et TTS.")

### Interpretation : Pipeline Multi-Modal

Ce pipeline illustre la puissance de combiner les modalites :

```
Texte simple          LLM           DALL-E         Vision          TTS
"paysage de    -->  Prompt    -->  Image     -->  Analyse   -->  Audio
 montagne"          enrichi        generee       poetique       narration
```

**Applications concretes** :

| Application | Pipeline |
|-------------|----------|
| **Podcast automatique** | Texte -> Resume -> TTS |
| **Accessibilite** | Image -> Description -> TTS |
| **Creation contenu** | Idee -> Image -> Post social |
| **Transcription meetings** | Audio -> Texte -> Resume -> Actions |

# Conclusion

## Resume des concepts

| Service | Direction | Code cle |
|---------|-----------|----------|
| **OpenAITextToImage** | Texte -> Image | `generate_image(description)` |
| **GPT-4V** | Image -> Texte | `ImageContent(uri=...)` |
| **OpenAIAudioToText** | Audio -> Texte | `get_text_content(audio)` |
| **OpenAITextToAudio** | Texte -> Audio | `get_audio_content(text)` |

## Points cles a retenir

1. **Interface unifiee** - Tous les services partagent le meme pattern
2. **Combiner les modalites** - Pipelines riches possibles
3. **Couts variables** - DALL-E 3 plus cher que Whisper
4. **GPT-4V pour l'analyse** - Pas de modele "vision" separe
5. **Qualite vs prix** - TTS-1-HD vs TTS-1, DALL-E 3 vs 2

## Exercices suggeres

1. **Assistant vocal** : Audio -> Texte -> LLM -> Audio
2. **Generateur de storyboard** : Script -> Serie d'images
3. **Traducteur visuel** : Image -> Description -> Traduction -> Image

## Pour aller plus loin

| Notebook | Contenu |
|----------|--------|
| [08-MCP](08-SemanticKernel-MCP.ipynb) | Model Context Protocol |
| [05-NotebookMaker](05-NotebookMaker.ipynb) | Agents multi-modaux |

---

**Navigation** : [<< 06-ProcessFramework](06-SemanticKernel-ProcessFramework.ipynb) | [Index](README.md) | [08-MCP >>](08-SemanticKernel-MCP.ipynb)